## Introduction

Author: Yingding Wang\
Created on: 17.11.2023

Example KFP v2 pipeline to use PVC https://pypi.org/project/kfp-kubernetes/

In [1]:
import sys

In [2]:
#!{sys.executable} -m pip install kfp==2.4.0 kfp-kubernetes==1.0.0 kfp-server-api==2.0.5 --upgrade --user

In [3]:
!{sys.executable} -m pip list | grep kfp

kfp                           2.4.0
kfp-kubernetes                1.0.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.5


In [4]:
#!{sys.executable} -m pip install --user --upgrade kfp==2.4.0 
#!{sys.executable} -m pip install --user --upgrade kfp-kubernetes==1.0.0

In [5]:
#!{sys.executable} -m pip install --user --upgrade kfp[kubernetes]==2.4.0

In [6]:
from kfp.dsl.pipeline_task import PipelineTask

def set_res_limit(task: PipelineTask, mem_req="200Mi", cpu_req="2000m", mem_lim="4000Mi", cpu_lim='4000m') -> PipelineTask:
    """set the resource limit for cpu and memory, no cpu and memory requirement sofar.
    should the limit is set to small, the Task Pod would be stopped by kubernetes with OOMKilled status.
    
    Args:
        task(PipelineTask): the KFP PipelineTask which need to be set the cpu and memory limits
        cpu_limit(str): the str representation of cpu limit e.g. '1' as one cpu time, '0.5' as 1/2 cpu time
        mem_limit(str): the str representation of memory limit e.g. '500M' for 500MB RAM
        
    Return:
        (PipelineTask): the PipelineTask with the desired limitations set
    """
    # return task.set_cpu_limit('1').set_memory_limit('500M')
    return task.set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)\
            .set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)

In [7]:
from kfp import dsl
from kfp import kubernetes
from kfp import compiler

@dsl.component(
base_image='python:3.10.13-bullseye'
)
def make_data():
    with open('/data/file.txt', 'w') as f:
        f.write('my data')

@dsl.component(
base_image='python:3.10.13-bullseye'
)
def read_data():
    with open('/reused_data/file.txt') as f:
        print(f.read())

@dsl.pipeline
def my_pipeline():
    pvc1 = kubernetes.CreatePVC(
        # can also use pvc_name instead of pvc_name_suffix to use a pre-existing PVC
        pvc_name_suffix='-my-pvc',
        access_modes=['ReadWriteOnce'],
        size='1Gi',
        storage_class_name='kubeflow-nfs-csi', # use the name of your available storage class      
    )

    task1 = make_data()
    task1 = set_res_limit(task=task1, cpu_req='500m', mem_req='200M')
    # normally task sequencing is handled by data exchange via component inputs/outputs
    # but since data is exchanged via volume, we need to call .after explicitly to sequence tasks
    task2 = read_data().after(task1)
    task2 = set_res_limit(task=task2, cpu_req='500m', mem_req='200M')
    
    kubernetes.mount_pvc(
         task1,
         pvc_name=pvc1.outputs['name'],
         mount_path='/data',
    )
    kubernetes.mount_pvc(
         task2,
         pvc_name=pvc1.outputs['name'],
         mount_path='/reused_data',
    )

    # wait to delete the PVC until after task2 completes
    delete_pvc1 = kubernetes.DeletePVC(
        pvc_name=pvc1.outputs['name']).after(task2)

In [8]:
compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path="./pvc_pipeline.yaml"
)

In [9]:
from kfp.client import Client
client = Client()
NAMESPACE = client.get_user_namespace()
print(NAMESPACE)

kubeflow-kindfor


/home/jovyan/.local/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


## Enable caching false is causing issue with PVC creationg
* https://github.com/kubeflow/pipelines/issues/10188

Note:
* this error will crash the kfp backend

You need to `kubectl -n kubeflow rollout restart deployment` to restart the container.

In [10]:
ENABLE_CACHING = True
# ENABLE_CACHING = False
EXPERIMENT_NAME = "demo"

run = client.create_run_from_pipeline_func(
    pipeline_func=my_pipeline,
    arguments = {},
    run_name="my pvc test",
    experiment_name = EXPERIMENT_NAME,
    namespace=NAMESPACE,
    enable_caching=ENABLE_CACHING,
)
run

RunPipelineResult(run_id=9b33874b-d8d4-4a0c-b530-da88a79e4cf6)